# 4. Analyse des Résultats et Maintenance

Ce notebook analyse les segments et simule la fréquence de mise à jour optimale du modèle.

**Objectifs :**
- Analyser les profils des segments
- Visualiser les résultats
- Simuler la dérive du modèle (ARI score)
- Déterminer la fréquence de mise à jour

**Auteur :** Thomas Mebarki  
**Date :** Janvier 2026

## 4.1 Configuration et imports

In [ ]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Modules du projet
from src.config import (
    PROCESSED_DATA_DIR, 
    MODELS_DIR,
    RAW_DATA_DIR,
    SEGMENT_NAMES,
    SEGMENT_COLORS,
    SEGMENT_DESCRIPTIONS,
    N_CLUSTERS,
    RANDOM_STATE
)
from src.models.clustering import CustomerSegmenter
from src.visualization.plots import (
    plot_rfm_boxplots, 
    plot_radar_chart, 
    plot_scatter_3d
)

sns.set_style('whitegrid')
%matplotlib inline

print("Modules importés avec succès")

## 4.2 Chargement des données segmentées

In [ ]:
# Charger les données avec segments
rfm_segmented = pd.read_parquet(PROCESSED_DATA_DIR / 'customers_rfm_segmented.parquet')

print(f"Shape: {rfm_segmented.shape}")
rfm_segmented.head()

In [ ]:
# Charger le modèle entraîné
segmenter = CustomerSegmenter.load(MODELS_DIR)
print(f"Modèle chargé: {segmenter.n_clusters} clusters")

## 4.3 Profil des segments

In [ ]:
# Statistiques par segment
segment_stats = rfm_segmented.groupby('segment_name').agg({
    'recency': ['mean', 'median', 'std'],
    'frequency': ['mean', 'median', 'std'],
    'monetary': ['mean', 'median', 'std'],
    'segment': 'count'
}).round(2)

segment_stats.columns = [
    'recency_mean', 'recency_median', 'recency_std',
    'frequency_mean', 'frequency_median', 'frequency_std',
    'monetary_mean', 'monetary_median', 'monetary_std',
    'count'
]

# Ajouter le pourcentage
segment_stats['pct'] = (segment_stats['count'] / segment_stats['count'].sum() * 100).round(1)

print("Profil des segments:")
segment_stats

In [ ]:
# Description des segments
print("\nDescription des segments:\n")
for segment_id, name in SEGMENT_NAMES.items():
    print(f"{name}:")
    print(f"  {SEGMENT_DESCRIPTIONS[segment_id]}")
    print()

## 4.4 Visualisations

In [ ]:
# Boxplots RFM par segment
fig = plot_rfm_boxplots(
    rfm_segmented[['recency', 'frequency', 'monetary']],
    rfm_segmented['segment'].values,
    save_path='../docs/rfm_boxplots.png'
)
plt.show()

In [ ]:
# Radar chart des profils
centers = segmenter.get_cluster_centers(scaled=False)
radar_fig = plot_radar_chart(centers, save_path='../docs/radar_chart.png')
radar_fig.show()

In [ ]:
# Visualisation 3D interactive
# Sous-échantillonner pour la performance
sample = rfm_segmented.sample(n=min(5000, len(rfm_segmented)), random_state=42)
scatter_fig = plot_scatter_3d(
    sample[['recency', 'frequency', 'monetary']],
    sample['segment'].values,
    save_path='../docs/scatter_3d.html'
)
scatter_fig.show()

## 4.5 Recommandations marketing par segment

In [ ]:
# Recommandations marketing
recommendations = {
    "Clients Récents": {
        "Priorité": "Haute",
        "Actions": [
            "Programme de bienvenue personnalisé",
            "Offre de deuxième achat (-10%)",
            "Email de suivi post-achat",
            "Recommandations de produits complémentaires"
        ]
    },
    "Clients Fidèles": {
        "Priorité": "Très haute",
        "Actions": [
            "Programme de fidélité avec récompenses",
            "Accès anticipé aux ventes privées",
            "Points de fidélité doublés",
            "Invitations à des événements exclusifs"
        ]
    },
    "Clients Dormants": {
        "Priorité": "Moyenne",
        "Actions": [
            "Campagne de réactivation par email",
            "Offre de retour attractive (-20%)",
            "Notification des nouveaux produits",
            "Sondage pour comprendre l'inactivité"
        ]
    },
    "Clients VIP": {
        "Priorité": "Critique",
        "Actions": [
            "Account manager dédié",
            "Livraison gratuite permanente",
            "Cadeaux personnalisés",
            "Support prioritaire"
        ]
    }
}

for segment, data in recommendations.items():
    print(f"\n{'='*50}")
    print(f"{segment.upper()}")
    print(f"Priorité: {data['Priorité']}")
    print(f"{'='*50}")
    for action in data['Actions']:
        print(f"  • {action}")

## 4.6 Simulation de maintenance du modèle

Nous simulons comment le modèle évolue dans le temps pour déterminer la fréquence de mise à jour optimale.

**Méthode :** Utiliser l'Adjusted Rand Index (ARI) pour mesurer la stabilité des segments.

In [ ]:
# Charger les données avec dates pour la simulation
try:
    transactions = pd.read_csv(RAW_DATA_DIR / 'data.csv', parse_dates=['order_purchase_timestamp'])
except:
    transactions = pd.read_csv(RAW_DATA_DIR / 'transactions_clean.csv', parse_dates=['order_purchase_timestamp'])

# Nettoyer
if 'Unnamed: 0' in transactions.columns:
    transactions = transactions.drop(columns=['Unnamed: 0'])

print(f"Transactions: {transactions.shape}")
print(f"Période: {transactions['order_purchase_timestamp'].min()} à {transactions['order_purchase_timestamp'].max()}")

In [ ]:
# Créer des snapshots hebdomadaires
transactions['week'] = transactions['order_purchase_timestamp'].dt.isocalendar().week + \
                       (transactions['order_purchase_timestamp'].dt.year - 2016) * 52

max_week = transactions['week'].max()
print(f"Nombre de semaines: {max_week}")

In [ ]:
def calculate_rfm_for_week(df, end_week):
    """
    Calcule les features RFM jusqu'à une semaine donnée.
    """
    subset = df[df['week'] <= end_week].copy()
    
    if len(subset) == 0:
        return None
    
    # Date de référence = fin de la semaine
    ref_date = subset['order_purchase_timestamp'].max()
    
    rfm = subset.groupby('customer_unique_id').agg({
        'order_purchase_timestamp': 'max',
        'order_id': 'nunique',
        'price': 'sum'
    })
    
    rfm.columns = ['last_purchase', 'frequency', 'monetary']
    rfm['recency'] = (ref_date - rfm['last_purchase']).dt.days
    
    return rfm[['recency', 'frequency', 'monetary']]

In [ ]:
# Simulation ARI sur différents intervalles
def simulate_model_drift(df, reference_week, test_weeks):
    """
    Simule la dérive du modèle en comparant les segments
    entre la semaine de référence et les semaines de test.
    """
    # Entraîner le modèle de référence
    ref_rfm = calculate_rfm_for_week(df, reference_week)
    if ref_rfm is None or len(ref_rfm) < 100:
        return None
    
    scaler = StandardScaler()
    ref_scaled = scaler.fit_transform(ref_rfm)
    
    kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=RANDOM_STATE, n_init=10)
    ref_labels = kmeans.fit_predict(ref_scaled)
    
    # Tester sur chaque semaine
    results = []
    for test_week in test_weeks:
        test_rfm = calculate_rfm_for_week(df, test_week)
        if test_rfm is None or len(test_rfm) < 100:
            continue
        
        # Clients communs
        common_clients = ref_rfm.index.intersection(test_rfm.index)
        if len(common_clients) < 50:
            continue
        
        # Prédire avec le modèle de référence
        test_scaled = scaler.transform(test_rfm.loc[common_clients])
        test_labels_pred = kmeans.predict(test_scaled)
        
        # Entraîner un nouveau modèle sur les données de test
        new_scaler = StandardScaler()
        test_full_scaled = new_scaler.fit_transform(test_rfm.loc[common_clients])
        new_kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=RANDOM_STATE, n_init=10)
        test_labels_new = new_kmeans.fit_predict(test_full_scaled)
        
        # Calculer l'ARI
        ari = adjusted_rand_score(test_labels_pred, test_labels_new)
        
        results.append({
            'reference_week': reference_week,
            'test_week': test_week,
            'weeks_gap': test_week - reference_week,
            'ari_score': ari,
            'n_common_clients': len(common_clients)
        })
    
    return pd.DataFrame(results)

In [ ]:
# Simuler la dérive
reference_week = 80  # Semaine de référence
test_weeks = range(reference_week + 1, min(reference_week + 30, int(max_week)))

print(f"Simulation de la dérive du modèle...")
print(f"Semaine de référence: {reference_week}")
print(f"Semaines de test: {list(test_weeks)[:5]}... (jusqu'à {max(test_weeks)})")

drift_results = simulate_model_drift(transactions, reference_week, test_weeks)

if drift_results is not None and len(drift_results) > 0:
    print(f"\nRésultats obtenus pour {len(drift_results)} semaines")
    drift_results.head(10)

In [ ]:
# Visualiser la dérive
if drift_results is not None and len(drift_results) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(drift_results['weeks_gap'], drift_results['ari_score'], 'b-o', linewidth=2, markersize=6)
    ax.axhline(y=0.8, color='green', linestyle='--', label='Seuil recommandé (0.8)')
    ax.axhline(y=0.6, color='orange', linestyle='--', label='Seuil d\'alerte (0.6)')
    
    ax.set_xlabel('Écart en semaines depuis l\'entraînement', fontsize=12)
    ax.set_ylabel('Score ARI (Adjusted Rand Index)', fontsize=12)
    ax.set_title('Évolution de la stabilité du modèle dans le temps', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig('../docs/model_drift.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Déterminer la fréquence de mise à jour
    threshold = 0.8
    weeks_above_threshold = drift_results[drift_results['ari_score'] >= threshold]['weeks_gap']
    
    if len(weeks_above_threshold) > 0:
        recommended_update = weeks_above_threshold.max()
        print(f"\nRecommandation: Réentraîner le modèle tous les {recommended_update} semaines")
        print(f"(pour maintenir un ARI >= {threshold})")
else:
    print("Pas assez de données pour la simulation de dérive")

## 4.7 Conclusion

### Résultats de la segmentation

| Segment | % | Caractéristiques | Action prioritaire |
|---------|---|-----------------|--------------------|
| Récents | 54% | Achat récent, 1 commande | Conversion en fidèles |
| Fidèles | 3% | Achats réguliers | Rétention |
| Dormants | 40% | Inactifs | Réactivation |
| VIP | 3% | Haute valeur | Premium |

### Maintenance du modèle

- **Fréquence de mise à jour recommandée** : ~3-4 mois
- **Indicateur de dérive** : Score ARI < 0.8
- **Action** : Réentraîner le modèle quand l'ARI descend sous le seuil

### Fichiers générés

- `docs/rfm_boxplots.png`
- `docs/radar_chart.png`
- `docs/scatter_3d.html`
- `docs/model_drift.png`